### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [87]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from sklearn import preprocessing
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from collections import Counter

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [88]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.SG = self.simple_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.SG = self.simple_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.SG = self.simple_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (1 - ((l_c ** 2 / l_s).sum(axis=1) +
                     (r_c ** 2 / r_s).sum(axis=1)) / (l_s + r_s)[0])

    def __entropy(self, l_c, l_s, r_c, r_s):
        return - ((np.log2((l_c / l_s) + 0.001) * l_c / l_s)
                  .sum(axis=1) +
                  (np.log2((r_c / r_s)+0.001) * r_c / r_s)
                  .sum(axis=1)) / (l_s + r_s)[0]

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s)[0]

    def simple_gini(self, y):
        return 1 - np.sum((np.bincount(y).astype(float) / y.size) ** 2)

    def simple_entropy(self, y):
        p = np.bincount(y).astype(float) / y.size
        return -np.sum(p * np.log(p + 0.001))

    def simple_misclass(self, y):
        return 1 - np.max(np.bincount(y).astype(float) / y.size)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        size = int(np.log2(n_feature))
        return feature_ids[:size]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)
        if len(r_border_ids) == 0:
            return np.inf, None
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if (y.shape[0] < self.min_samples_split or
                (self.max_depth is not None and depth >= self.max_depth) or
                self.sufficient_share <= float(np.bincount(y).max()) / y.size):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        ids = self.get_feature_ids(x.shape[1])
        ths = np.array([self.__find_threshold(x[:, i], y) for i in ids])
        th_ind = np.argmin(ths[:, 0])
        if th_ind is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        th = ths[th_ind][1]
        if th is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, ids[th_ind], th)
        if (x_l.shape[0] == 0 or x_r.shape[0] == 0):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        self.feature_importances_[ids[th_ind]] += self.SG(y) - \
            (self.SG(y_l) * y_l.size + self.SG(y_r) * y_r.size) / y.size
        self.tree[node_id] = (self.NON_LEAF_TYPE, ids[th_ind], th)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ = np.array(self.feature_importances_)
        s = self.feature_importances_.sum()
        self.feature_importances_ = self.feature_importances_ / s

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [89]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [90]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [91]:
%time clf.fit(X_train, y_train)

CPU times: user 2.71 ms, sys: 0 ns, total: 2.71 ms
Wall time: 2.59 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [92]:
%time my_clf.fit(X_train, y_train)

CPU times: user 23.5 ms, sys: 0 ns, total: 23.5 ms
Wall time: 22.9 ms


## Проверка качества работы на wine

In [93]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9407407407407408

In [94]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9407407407407408

In [95]:
clf.feature_importances_

array([0.03668117, 0.        , 0.        , 0.01519649, 0.        ,
       0.        , 0.19184787, 0.        , 0.        , 0.35587373,
       0.01424671, 0.        , 0.38615403])

In [96]:
my_clf.feature_importances_

array([0.32398312, 0.29482254, 0.        , 0.        , 0.02800871,
       0.        , 0.12680612, 0.        , 0.        , 0.12837814,
       0.        , 0.        , 0.09800137])

## Подготовка данных Speed Dating Data 

In [40]:
df = pd.read_csv('Speed Dating Data.csv', encoding='latin1')

In [41]:
pd.set_option('display.max_columns', 500)

In [42]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,2,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,3,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,4,14.0,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,1,7.0,8.0,9.0,8.0,9

In [43]:
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.dropna(subset=['age'])
df = df.drop(['undergra'], axis=1)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df = df.drop(['income'], axis=1)
df = df.dropna(subset=['date'])
df = df.drop(['sinc5_1', 'fun5_1', 'sinc1_s', 'fun3_s',
              'fun7_2', 'amb3_2', 'num_in_3', 'amb1_3',
              'fun7_3', 'shar7_3', 'attr4_3', 'sinc4_3',
              'fun4_3', 'amb4_3', 'shar4_3', 'intel2_3',
              'attr3_3', 'intel3_3', 'fun3_3', 'amb3_3',
              'attr5_3', 'sinc5_3', 'intel5_3', 'attr3_s',
              'attr2_2', 'sinc5_2', 'sinc7_3', 'intel7_3'],
             axis=1)

Nan = -1, выделим категориальные признаки

In [44]:
df = df.fillna(-1)

In [45]:
le = preprocessing.LabelEncoder()
for i in df.columns:
    if (df[i].dtype == 'str' or df[i].dtype == 'object'):
        df[i] = le.fit_transform(list(df[i]))

In [46]:
df.head()

,iid,gender,wave,pid,match,int_corr,samerace,age,field,field_cd,mn_sat,tuition,race,imprace,imprelig,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,intel5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,intel1_s,fun1_s,amb1_s,shar1_s,sinc3_s,intel3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,attr5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,attr1_3,sinc1_3,intel1_3,fun1_3,shar1_3,attr7_3,amb7_3,intel4_3,attr2_3,sinc2_3,fun2_3,amb2_3,shar2_3,sinc3_3,fun5_3,amb5_3
0,1,0,1,11.0,0,0.14,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
1,1,0,1,12.0,0,0.54,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
2,1,0,1,13.0,1,0.16,1,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,-1.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
3,1,0,1,14.0,1,0.61,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
4,1,0,1,15.0,1,0.21,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0

In [47]:
for i in df.columns:
    if len(df[i].unique()) < 30:
        df[i] = df[i].astype('category')

In [48]:
y = np.array(df['match'])
y

array([0, 0, 1, ..., 0, 0, 0])

In [49]:
df1 = df.drop(['match'], axis=1)
df1.head()

,iid,gender,wave,pid,int_corr,samerace,age,field,field_cd,mn_sat,tuition,race,imprace,imprelig,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,intel5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,intel1_s,fun1_s,amb1_s,shar1_s,sinc3_s,intel3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,attr5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,attr1_3,sinc1_3,intel1_3,fun1_3,shar1_3,attr7_3,amb7_3,intel4_3,attr2_3,sinc2_3,fun2_3,amb2_3,shar2_3,sinc3_3,fun5_3,amb5_3
0,1,0,1,11.0,0.14,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
1,1,0,1,12.0,0.54,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
2,1,0,1,13.0,0.16,1,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,-1.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
3,1,0,1,14.0,0.61,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,20.0,20.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0
4,1,0,1,15.0,0.21,0,21.0,123,1.0,0,0,4.0,2.0,4.0,2.0,7.0,1.0,300,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,1,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19.44,16.67,13.89,22.22,11.11,16.67,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,7.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,15.0,2

In [50]:
X = np.array(df1)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [55]:
my_clf = MyDecisionTreeClassifier(min_samples_split=3, max_features='sqrt', max_depth=8)
clf = DecisionTreeClassifier(min_samples_split=3, max_features='sqrt', max_depth=8)

## Проверка скорости работы на Speed Dating Data 

In [56]:
%time clf.fit(X_train, y_train)

CPU times: user 114 ms, sys: 8 µs, total: 114 ms
Wall time: 111 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [57]:
%time my_clf.fit(X_train, y_train)

CPU times: user 413 ms, sys: 3.99 ms, total: 417 ms
Wall time: 417 ms


## Проверка качества работы на Speed Dating Data

In [58]:
print(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro'))
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)

0.5554625040169051


0.8490909090909091

In [59]:
print(f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro'))
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

0.5813839201882623


0.8460606060606061

## Задание 3

In [63]:
clf.feature_importances_[:10]

array([0.        , 0.        , 0.        , 0.02749137, 0.00274164,
       0.00267749, 0.00991612, 0.        , 0.00344663, 0.00645232])

In [64]:
my_clf.feature_importances_[:10]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01735692, 0.        , 0.        , 0.        ])

## Задание 4

In [65]:
clf = RandomForestClassifier()
parametres = {"max_depth": range(1, 20),
              "min_samples_split": range(2, 20),
              "n_estimators": range(1, 100),
              "criterion": ['gini', 'entropy']}
random_search = RandomizedSearchCV(clf, parametres, cv=5)
random_search.fit(X_train, y_train)
print (random_search.best_params_)

{'n_estimators': 51, 'min_samples_split': 12, 'max_depth': 18, 'criterion': 'entropy'}
